In [1]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=bf49202010cd5fd6ee11ed8d90fa82c6ec716e098180fa48bcdef27c824f67b0
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [2]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import LinearSVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


trainfile = r'/gdrive/My Drive/CIS508A3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS508A3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]
(173836, 596)
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...    

In [0]:

#Copy Train data excluding target
X_train = train_data.iloc[:, :-1].copy()
Y_train = train_data["QuoteConversion_Flag"]
X_test = test_data.iloc[:, :-1].copy()


In [0]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)
clf_predict=clf.predict(X_test)

In [6]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(70,200,5),'max_depth': range(20,70,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, Y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,Y_train)
clf_predict = clf.predict(X_test)


{'min_samples_split': 160, 'max_depth': 66}


In [7]:
#Get Class Prediction as a data frame with header as Prediction
pred_probability=pd.DataFrame(clf.predict(X_test),columns=["QuoteConversion_Flag"])

pred_probability.head()
#Write into a file with actual prediction and corresponding probability
pd.concat([X_test['QuoteNumber'],pred_probability],axis=1).to_csv("/gdrive/My Drive/CIS508A3/DecisionTreeresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/CIS508A3/DecisionTreeresults.csv')
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_train, Y_train)
rfc_predict=rfc.predict(X_test)

In [9]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,200,20),'min_samples_split' : range(50,100,10),'max_depth': range(1,50,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, Y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,Y_train)
rfc_predict = rfc.predict(X_test)

{'n_estimators': 190, 'min_samples_split': 80, 'max_depth': 27}


In [10]:
#Get Class Prediction as a data frame with header as Prediction
pred_probability=pd.DataFrame(rfc.predict(X_test),columns=["QuoteConversion_Flag"])

pred_probability.head()
#Write into a file with actual prediction and corresponding probability
pd.concat([X_test['QuoteNumber'],pred_probability],axis=1).to_csv("/gdrive/My Drive/CIS508A3/RandomForestresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/CIS508A3/RandomForestresults.csv')
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [11]:
#Gradient Boosting ============================================================
search_grid={'n_estimators':[50,60,70,80],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(X_train, Y_train)
abc_predict=abc.predict(X_test)

#Randomized Search for hyperparameter tuning
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, Y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train,Y_train)
abc_predict = abc.predict(X_test)

{'n_estimators': 50, 'learning_rate': 0.1}


In [12]:
#Get Class Prediction as a data frame with header as Prediction
pred_probability=pd.DataFrame(abc.predict(X_test),columns=["QuoteConversion_Flag"])

pred_probability.head()
#Write into a file with actual prediction and corresponding probability
pd.concat([X_test['QuoteNumber'],pred_probability],axis=1).to_csv("/gdrive/My Drive/CIS508A3/GBresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/CIS508A3/GBresults.csv')
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [17]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(Y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.7)
X_res, y_res = sm.fit_resample(X_train, Y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 36916})


In [18]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [RandomForestClassifier(**grid_parm_rfc), DecisionTreeClassifier(**grid_parm),GradientBoostingClassifier(**grid_parm_abc)]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestClassifier]
    fold  0:  [0.93160525]
    fold  1:  [0.93370215]
    fold  2:  [0.93298532]
    fold  3:  [0.93084371]
    ----
    MEAN:     [0.93228411] + [0.00112229]
    FULL:     [0.93228411]

model  1:     [DecisionTreeClassifier]
    fold  0:  [0.92696529]
    fold  1:  [0.93106987]
    fold  2:  [0.92785437]
    fold  3:  [0.92896979]
    ----
    MEAN:     [0.92871483] + [0.00153399]
    FULL:     [0.92871484]

model  2:     [GradientBoostingClassifier]
    fold  0:  [0.93615597]
    fold  1:  [0.93637905]
    fold  2:  [0.93646544]
    fold  3:  [0.93445768]
    ----
    MEAN:     [0.93586454] + [0.00082006]
    FULL:     [0.93586455]



In [0]:
#STACKING - CONSTRUCT Random Forest MODEL==============================
model = RandomForestClassifier()
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)



In [20]:
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(model.predict(S_Test),columns=["QuoteConversion_Flag"])

pred_Probability.head()


,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0


In [21]:
#Write into a file with actual prediction and corresponding probability
pd.concat([X_test['QuoteNumber'],pred_Probability],axis=1).to_csv("/gdrive/My Drive/CIS508A3/Stacking3_2results.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/CIS508A3/Stacking3_2results.csv')
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0
